### MELD

In this notebook we'll merge results from the two techniques used to collect metadata from the phot archive

- **metadata** extraction with exiftool
- **ocr** with tessaract

We'll discuss what we obtained and we'll eventually merge the data to obtain the final dataset

In [30]:
import os
import pandas as pd
import numpy as np

from PIL import Image

In [31]:
datasets_directory = '../datasets'

df_ocr = pd.read_csv(f'{datasets_directory}/ocr.csv', index_col=0)
df_metadata = pd.read_csv(f'{datasets_directory}/metadata.csv', index_col=0)

In [32]:
print('Dataframe without OCR')
print(df_metadata.info())

Dataframe without OCR
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16874 entries, 0 to 16873
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hash        16874 non-null  object
 1   image_name  16874 non-null  object
 2   camera      12422 non-null  object
 3   date_time   16771 non-null  object
 4   date        16771 non-null  object
 5   time        16771 non-null  object
 6   moon        16771 non-null  object
dtypes: object(7)
memory usage: 1.0+ MB
None


In [33]:
print('Dataframe with OCR')
print(df_ocr.info())

Dataframe with OCR
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16874 entries, 0 to 16873
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  16874 non-null  object
 1   camera      16337 non-null  object
 2   date_time   16771 non-null  object
 3   date        16862 non-null  object
 4   time        16862 non-null  object
 5   moon        16771 non-null  object
dtypes: object(6)
memory usage: 922.8+ KB
None
